In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
#%cd /content/gdrive/MyDrive/Terpli_Code

In [82]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mlp
from scipy import stats
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

# Import data

In [83]:
clustered = pd.read_excel('mixed_clustered_Jan5.xlsx',converters={
       'cluster_terpene_only':str, 'terpenefirst_sub_cluster':str,
       'terpenefirst_sub_cluster2':str, 'terpenefirst_multilayer_label':str,
       'terpenefirst_multilayer_cluster_full_label':str,
       'terpenefirst_multilayer_cluster_6digit_label':str,
       'terpenefirst_cluster_adjust_label_2':str, 
       'cluster_cannabi_only.1':str, 'cluster_terpene_only.1':str,
       'cluster_other_ratios_only':str
       
                                                                    })

In [84]:
#'mixed_clustered_20_dec.xlsx'
#'concentrates_clustered_20_dec.xlsx'
#'plant_clustered_20_dec.csv'

In [85]:
%%time
## cannabis profile data after cleaning
data = pd.read_csv('Jan2022_Cannabis-Profiles_Data Science_cleaned.csv')
## original data with effects
raw =pd.read_csv('Cannabis-Profiles_Jan5.csv')

## customer reviews
## there are some products that are not in our cannabis profile dataset
#reviews = pd.read_excel('Cannabis-Reviews_Worksheet.xlsx', sheet_name='Active Reviews')

new_reviews =  pd.read_excel('Terpli Review Organization.xlsx',sheet_name='Sheet1', 
                             skiprows = range(168, 224),
                             parse_dates=[1])
## pairwised similarity score,  calculated by using weighted cosine distance/Euclidian distance
## of groups of features (terpenes perc, vol, cannabis perc, vol, terpene total vol )

similarity  = pd.read_csv('product_similarity_score_sorted.csv')

#demographics = pd.read_csv('All Users From DB - Sheet1.csv')

CPU times: user 14.6 s, sys: 2.49 s, total: 17 s
Wall time: 18.1 s


In [86]:
data.columns

Index(['Unnamed: 0', 'Strain_Name', 'Batch_ID', 'A_Pinene', 'Distributor',
       'Strain', 'MetricID', 'Type', 'Distributor_Address',
       'Distributor_City_State_Zip', 'B_Caryophyllene', 'Producer', 'Brand',
       'Date_Recieved', 'Matrix', 'Producer_City_State_Zip',
       'Producer_Lic_Number', 'terpinolene', 'Producer_Address', 'Sample_ID',
       'Lab_ID', 'Total_Terpene_Volume', 'A_Humulene', 'Date_Harvested',
       'B_Myrcene', 'dataOrigin', 'Ocimene', 'Date_Collected', 'D_Limonene',
       'brandPK', 'B_Pinene', 'Distributor_Lic#', 'Linalool',
       'Date_Test_Completed', 'PrimaryKey', 'THC_Pct', 'CBD_Pct', 'CBG_Pct',
       'CBN_Pct', 'CBC_Pct', 'CBDV_Pct', 'THCV_Pct', 'CBL_Pct', 'D8_Pct',
       'Total_Cannabinoid_Pct', 'Nerolidol', 'A-Bisabalol', 'Other',
       'B_Myrcene_Pct_Total', 'B_Caryophyllene_Pct_Total',
       'D_Limonene_Pct_Total', 'A_Pinene_Pct_Total', 'B_Pinene_Pct_Total',
       'A_Humulene_Pct_Total', 'terpinolene_Pct_Total', 'Linalool_Pct_Total',
     

In [87]:
#reviews['Batch ID'] = reviews['Batch ID'].astype(str)


In [88]:
experiences_col = [
       'Art / Music', 'Mood Boosting', 'Exercise', 'Focus', 'Gaming',
       'Getting Stoned', "I didn't like it", 'Creativity', 'Mornings',
       'Pain Relief', 'Relaxing', 'Romancing', 'Sleeping', 'Socializing']
effects_col = [  'Stimulated', 'Calm', 'Creative', 'Energized', 'Euphoric',
       'Focused', 'Relaxed', 'Sleepy', 'Uplifted', 'Sedated', 'othereffects_',
       'Spacey', 'Boring', 'Paranoia', 'Dry eyes', 'Headache', 'Anxious',
       'Dizzy', 'Cottonmouth', 'Sluggish', 'Hazy', 'Annoyed', 'Aroused',
       'Emotional', 'Antisocial']

experiences_effects_col = experiences_col+effects_col


## ratings statistics

In [89]:
new_reviews['username'].value_counts()

kasper0017@gmail.com                           122
faded-cheese-032@anonymous.appuser.io           30
chris.10sne1@gmail.com                           5
fearful-invention-895@anonymous.appuser.io       3
vocal-distribution-255@anonymous.appuser.io      2
vishnubh@marshall.usc.edu                        1
eblasser91@gmail.com                             1
gannott@usc.edu                                  1
zmattioli@gmail.com                              1
rthakkar@usc.edu                                 1
Name: username, dtype: int64

In [90]:
new_reviews['rating'].describe()

count    165.000000
mean       4.200000
std        0.877913
min        1.000000
25%        4.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: rating, dtype: float64

In [91]:
new_reviews.groupby('username')['rating'].mean().sort_values()

username
zmattioli@gmail.com                            3.000000
vocal-distribution-255@anonymous.appuser.io    3.500000
fearful-invention-895@anonymous.appuser.io     3.666667
chris.10sne1@gmail.com                         4.000000
eblasser91@gmail.com                           4.000000
rthakkar@usc.edu                               4.000000
vishnubh@marshall.usc.edu                      4.000000
kasper0017@gmail.com                           4.155738
faded-cheese-032@anonymous.appuser.io          4.551724
gannott@usc.edu                                5.000000
Name: rating, dtype: float64

In [92]:
new_reviews[new_reviews['rating']<=2]

,username,Date,Batch ID,Matrix,type,Strain Name,Brand,notes,smokeType,timeOfDay,...,Headache,Anxious,Dizzy,Cottonmouth,Sluggish,Hazy,Annoyed,Aroused,Emotional,Antisocial
51,kasper0017@gmail.com,2020-05-17 22:37:42,RM4H36.SD,Plant,Flower - Cured,Sour Diesel,The Plug LA,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
64,kasper0017@gmail.com,2020-05-18 16:58:48,RM4H34.TRIFI,Plant,Flower - Cured,Tri-Fi Cookies,The Plug LA,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
66,kasper0017@gmail.com,2020-05-20 01:42:08,RLR034,Concentrates & Extracts,Diamonds,Purple Sunset OG,Raw Garden,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
97,kasper0017@gmail.com,2020-12-27 19:51:09,7916,Concentrates & Extracts,Live Resin,Purgatory,Everest,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
110,kasper0017@gmail.com,2021-04-16 20:03:50,RM4H64.GUSH,Plant,Flower - Cured,White Gushers,The Plug,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
115,fearful-invention-895@anonymous.appuser.io,2021-04-22 17:43:04,C1.21.07,Plant,Flower - Cured,Grease Monkey,Seven Leaves,NaN,NaN,NaN,...,0,1,0,0,0,1,0,0,0,1
143,vocal-distribution-255@anonymous.appuser.io,2021-05-13 15:43:11,1600ADUBSRM132021,Plant,Flower - Cured,ADUB,MJ Ultra,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
164,kasper0017@gmail.com,NaT,EAR0-CP42J13-D077,Concentrates & Extracts,Vape Cartridge,Earth OG,Kurvana,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


### rating normalization

Some customers tend to give higher ratings and some give lower. To normalize this effect, we use deduct the average rating of one customer from his ratings.



In [93]:
## normalize the ratings
#df["value"] = df.groupby("name").transform(lambda x: x.fillna(x.mean()))
new_reviews['rating_mean'] = new_reviews.groupby('username')['rating'].transform('mean')


In [94]:
new_reviews['rating_mean']

0      4.155738
1      4.155738
2      4.155738
3      4.155738
4      4.155738
         ...   
162    4.551724
163    4.155738
164    4.155738
165    4.155738
166    4.155738
Name: rating_mean, Length: 167, dtype: float64

In [95]:
new_reviews['rating_norm'] = new_reviews['rating']-round(new_reviews['rating_mean'],2)
new_reviews['rating_norm']

0      0.84
1      0.84
2      0.84
3      0.84
4      0.84
       ... 
162    0.45
163   -0.16
164   -2.16
165   -0.16
166   -0.16
Name: rating_norm, Length: 167, dtype: float64

In [96]:
new_reviews['rating_norm'].describe()

count    165.000000
mean      -0.002909
std        0.849609
min       -3.160000
25%       -0.160000
50%       -0.160000
75%        0.840000
max        1.500000
Name: rating_norm, dtype: float64

The number of 'negative reviews' using different threshold.

In [97]:
len(new_reviews[new_reviews['rating_norm']<=0])

95

In [98]:
len(new_reviews[new_reviews['rating_norm']<=-1])

27

In [99]:
len(new_reviews[new_reviews['rating_norm']<=-2])

7

In [100]:
len(new_reviews[new_reviews['rating_norm']<=-3])

1

We can use the threshold -1 to define the 'negative reviews'


## remove negative review products

In [101]:
new_reviews['review_sentiment'] = np.where(new_reviews['rating_norm']>-2 , 'Positive', 'Negative'
                                          )

In [102]:
new_reviews['review_sentiment'].value_counts()

Positive    158
Negative      9
Name: review_sentiment, dtype: int64

In [103]:
good_reviews = new_reviews[new_reviews['review_sentiment']=='Positive']

In [104]:
reviews = good_reviews

# Similarity Score Based 

idea: for multiple products being reviewed as a certain experiences, find the 'clustroid'. Unlike 'centroid', which is the average, a non-existed virtual center, clustroid is an existing point, which is most close to all the other points.
    
User enter 0-2 effects and 0-2 experiences.

    
    




In [105]:
user = 'kasper0017@gmail.com'
user_mean = reviews[reviews['username'] == user].groupby('username')[experiences_col].mean().T

user_mean = user_mean.reset_index().rename(columns = {'index':'experiences',
                                                     user: 'score'})
user_mean.index = user_mean.index.set_names(['index'])
user_mean = user_mean.sort_values('score',ascending = False)
user_mean.head(3)['experiences'].values

array(['Sleeping', 'Focus', 'Socializing'], dtype=object)

In [106]:
def most_reviewed_experiences(user):
    #user = 'kasper0017@gmail.com'
    user_mean = reviews[reviews['username'] == user].groupby('username')[experiences_col].mean().T

    user_mean = user_mean.reset_index().rename(columns = {'index':'experiences',
                                                         user: 'score'})
    user_mean.index = user_mean.index.set_names(['index'])
    user_mean = user_mean.sort_values('score',ascending = False)
    result = user_mean.head(3)['experiences'].values
    return result
def most_reviewed_effects(user):
    #user = 'kasper0017@gmail.com'
    user_mean = reviews[reviews['username'] == user].groupby('username')[effects_col].mean().T

    user_mean = user_mean.reset_index().rename(columns = {'index':'effects',
                                                         user: 'score'})
    user_mean.index = user_mean.index.set_names(['index'])
    user_mean = user_mean.sort_values('score',ascending = False)
    result = user_mean.head(3)['effects'].values
    return result

In [107]:
most_reviewed_experiences('kasper0017@gmail.com')

array(['Sleeping', 'Focus', 'Socializing'], dtype=object)

In [108]:
most_reviewed_effects('kasper0017@gmail.com')

array(['Euphoric', 'Focused', 'Relaxed'], dtype=object)

In [109]:
most_reviewed_experiences('faded-cheese-032@anonymous.appuser.io')

array(['Mood Boosting', 'Relaxing', 'Getting Stoned'], dtype=object)

In [110]:
most_reviewed_experiences('chris.10sne1@gmail.com')

array(['Focus', 'Sleeping', 'Art / Music'], dtype=object)

In [111]:
most_reviewed_experiences('vocal-distribution-255@anonymous.appuser.io')

array(['Getting Stoned', 'Relaxing', 'Art / Music'], dtype=object)

In [112]:
most_reviewed_experiences('vishnubh@marshall.usc.edu')

array(['Art / Music', 'Mood Boosting', 'Exercise'], dtype=object)

## User - Experiences

In [113]:
user = 'kasper0017@gmail.com'
effects = ['Euphoric','Focused']
experience = ['Sleeping']
#experience = []
effects_experiences = effects+experience
effects_experiences 

['Euphoric', 'Focused', 'Sleeping']

## Try 3 rules of selecting base products

In [114]:
### any match (any = 1)
new_reviews[new_reviews[effects_experiences].isin([1]).any(axis=1)][effects_experiences]


,Euphoric,Focused,Sleeping
0,1,0,1
1,1,0,0
2,1,0,0
3,0,0,1
4,1,0,0
...,...,...,...
159,1,1,0
161,0,1,0
163,1,1,0
164,1,0,0


In [115]:
### approximate match (any = 1 in experiences and any = 1 in effects)

#any = 1 in effects
effect_temp = set(list(new_reviews[new_reviews[effects].isin([1]).any(axis=1)]['Batch ID']))

#any = 1 in expeirences
experience_temp =set(list(new_reviews[new_reviews[experience].isin([1]).any(axis=1)]['Batch ID']))

# the intersection of the two above
intersect = list(experience_temp.intersection(experience_temp))
new_reviews[new_reviews['Batch ID'].isin(intersect)][effects_experiences]





,Euphoric,Focused,Sleeping
0,1,0,1
3,0,0,1
4,1,0,0
5,1,1,1
8,0,0,1
10,1,0,0
12,1,0,1
14,1,1,0
16,0,1,1
17,0,1,1


In [116]:
print(len(intersect))

17


In [117]:
## exact match (all = 1)

new_reviews[new_reviews[effects_experiences].isin([1]).all(axis=1)][effects_experiences]

,Euphoric,Focused,Sleeping
5,1,1,1


Use approximate match or exact match? depends on the number of review of the user. Right now we can use approximate match to start. 


Any  = 1 in the effects list and any = 1 in the experiences list. 


### below was a example of simplest rule

In [118]:
new_reviews['Batch ID'] = new_reviews['Batch ID'].astype(str)

In [119]:
new_reviews[new_reviews['Batch ID'].isin(intersect)][effects_experiences]
product_list = intersect

In [120]:
product_list = pd.Series(product_list)

In [121]:
product_list

0                   769
1          LR-FB-CC-177
2       HS008-HL-042420
3                POG.NH
4        25PT200930d9I1
5           8jdi3kd0kq$
6     ST-CLR-PUH-090720
7             RM3H62.LC
8          HB200422A-SA
9          LAKKB021420A
10            21040521C
11                BO.NH
12            KSS040720
13      HS019-HL-063020
14                 7916
15           KWI5262020
16              GT2500B
dtype: object

In [122]:
similarity['initial_product'] = similarity['initial_product'].astype(str)
similarity['product_ID'] = similarity['product_ID'].astype(str)

In [123]:
similarity[(similarity['initial_product'] =='ST-CLR-PUH-090720')]

,Unnamed: 0,Unnamed: 0_cannabis_perc,initial_product,product_ID,product_cos_distance_cannabis_perc,product_cos_similarity_cannabis_perc,Unnamed: 0_terpenes_perc,product_cos_distance_terpenes_perc,product_cos_similarity_terpenes_perc,Unnamed: 0_cannabis_vol,product_cos_distance_cannabis_vol,product_cos_similarity_cannabis_vol,Unnamed: 0_terpenes_vol,product_cos_distance_terpenes_vol,product_cos_similarity_terpenes_vol,Unnamed: 0_terpenes_cannabis_vol,product_cos_distance_terpenes_cannabis_vol,product_cos_similarity_terpenes_cannabis_vol,weighted_w_tcvol


In [124]:
product_info = data[data['Batch_ID'].isin(product_list)]
product_info

,Unnamed: 0,Strain_Name,Batch_ID,A_Pinene,Distributor,Strain,MetricID,Type,Distributor_Address,Distributor_City_State_Zip,...,THC_Pct_Pct_Total,CBD_Pct_Pct_Total,CBG_Pct_Pct_Total,CBN_Pct_Pct_Total,CBC_Pct_Pct_Total,CBDV_Pct_Pct_Total,THCV_Pct_Pct_Total,CBL_Pct_Pct_Total,D8_Pct_Pct_Total,Total_Terpene_Volume_pct
788,801,Black Orchid,BO.NH,0.71,PHC Facilities. Inc.,black orchid,0,Flower - Cured,1425 Long Beach Avenue,Los Angeles. CA 90021,...,0.970376,0.001381,0.024102,0.000000,0.004142,0.0,0.000000,0.0,0.0,0.01210
903,941,Kushberry Cheesecake,LAKKB021420A,0.14,Modern Determinism. LLC,kushberry cheesecake,0,Flower - Cured,959 East Pico Blvd,Los Angeles. CA 90021,...,0.983728,0.001901,0.014371,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.01731
1032,1076,Fog Berry,LR-FB-CC-177,0.20,Eel River Dry Farms. Inc,fog berry,1A4060300004D5A000000481,Live Resin,4300 West End RD. Bldg A. Arcata. CA 95521,0,...,0.942467,0.002557,0.031864,0.000287,0.016841,0.0,0.005985,0.0,0.0,0.04537
1094,1140,Sour Sandia Hash Rosin,KSS040720,1.97,Broadway Alliance,sour sandia hash rosin,1A4060300005399000003116,Wax,2408 S Broadway. Los Angeles. CA 90007,0,...,0.939948,0.001707,0.058345,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.03477
1181,1231,Gorilla Grease Badder,HS008-HL-042420,0.51,The Hive Laboratory,gorilla grease badder,1A406030000189D000002770,Other,949 Highland Way. Grovery Beach. CA 93433,0,...,0.981626,0.001480,0.015403,0.000000,0.001491,0.0,0.000000,0.0,0.0,0.03540
1424,1492,Wonka Hash Rosin,KWI5262020,2.02,Broadway Alliance,wonka hash rosin,1A4060300005399000003840,Rosin,2408 S Broadway. Los Angeles. CA 90007,0,...,0.971671,0.001787,0.026178,0.000365,0.000000,0.0,0.000000,0.0,0.0,0.03092
1507,1579,Honey Boo,HB200422A-SA,13.40,SLO Cultivation Inc.,honey boo,1A4060300003736000002384,Other,1269 Marie St. #2. Mendota. CA 93640-2656,0,...,0.949694,0.001842,0.039252,0.000000,0.005710,0.0,0.003502,0.0,0.0,0.05754
1532,1608,Tangerine Badder,HS019-HL-063020,5.00,The Hive Laboratory,tangerine badder,1A406030000189D000003119,Other,949 Highland Way . Grover Beach. CA. 93433,0,...,0.977235,0.001311,0.014161,0.001942,0.004745,0.0,0.000607,0.0,0.0,0.02941
1606,1687,Falcon 9,GT2500B,0.20,PHC Facilities,falcon 9,1A4060300008471000001089,Batter,1425 Long Beach Ave,0,...,0.978120,0.001521,0.020359,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.01740
2063,2233,Platinum O.G.,POG.NH,0.10,0,platinum o.g.,0,Flower - Cured,0,0,...,0.962864,0.003048,0.034088,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00904


In [125]:
# features = ['THC %', 'CBD %', 'CBG %', 'CBN %', 'CBC %', 'CBDV %', 'THCV %',
#        'CBL %','D8 %','Total Cannabinoid %','B_Myrcene_% of Total',
#        'B_Caryophyllene_% of Total', 'D_Limonene_% of Total',
#        'A_Pinene_% of Total', 'B_Pinene_% of Total', 'A_Humulene_% of Total',
#        'terpinolene_% of Total', 'Linalool_% of Total', 'Ocimene_% of Total',
#        'Nerolidol_% of Total', 'A-Bisabalol_% of Total', 'Other_% of Total',]

# cannabis_vol = raw_trimmed[[ 'THC %', 'CBD %', 'CBG %', 'CBN %', 'CBC %', 'CBDV %', 'THCV %',
#        'CBL %','D8 %','Total Cannabinoid %']]

# terpenes_perc =  raw_trimmed[['B_Myrcene_% of Total',
#        'B_Caryophyllene_% of Total', 'D_Limonene_% of Total',
#        'A_Pinene_% of Total', 'B_Pinene_% of Total', 'A_Humulene_% of Total',
#        'terpinolene_% of Total', 'Linalool_% of Total', 'Ocimene_% of Total',
#        'Nerolidol_% of Total', 'A-Bisabalol_% of Total', 'Other_% of Total']]

# cannabis_perc =  raw_trimmed[[ 'THC %_% of Total', 'CBD %_% of Total',
#        'CBG %_% of Total', 'CBN %_% of Total', 'CBC %_% of Total',
#        'CBDV %_% of Total', 'THCV %_% of Total', 'CBL %_% of Total',
#        'D8 %_% of Total']]

# terpenes_vol =raw_trimmed[[ 'B_Myrcene', 'B_Caryophyllene',
#        'D_Limonene', 'A_Pinene', 'B_Pinene', 'A_Humulene', 'terpinolene',
#        'Linalool', 'Ocimene', 'Nerolidol', 'A-Bisabalol', 'Other',
#        'Total Terpene Volume']]

# test_vol =raw_trimmed[[ 
#        'Total Terpene Volume %','Total Cannabinoid %']]

In [126]:
similarity['product_ID']

0              GHFGG40930
1             LDRBT0407PR
2               210000445
3                   RV450
4                 2000091
                ...      
13259543    051420AFLV159
13259544        S010720-6
13259545      PB090821-Sb
13259546       FFSD0527MP
13259547        S010720-6
Name: product_ID, Length: 13259548, dtype: object

## Find clustroid

In [127]:
similarity[(similarity['initial_product'].isin(product_list))]

,Unnamed: 0,Unnamed: 0_cannabis_perc,initial_product,product_ID,product_cos_distance_cannabis_perc,product_cos_similarity_cannabis_perc,Unnamed: 0_terpenes_perc,product_cos_distance_terpenes_perc,product_cos_similarity_terpenes_perc,Unnamed: 0_cannabis_vol,product_cos_distance_cannabis_vol,product_cos_similarity_cannabis_vol,Unnamed: 0_terpenes_vol,product_cos_distance_terpenes_vol,product_cos_similarity_terpenes_vol,Unnamed: 0_terpenes_cannabis_vol,product_cos_distance_terpenes_cannabis_vol,product_cos_similarity_terpenes_cannabis_vol,weighted_w_tcvol
235,2502732,0,BO.NH,BO.NH,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
289,973738,0,GT2500B,GT2500B,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
514,5355280,0,HS008-HL-042420,HS008-HL-042420,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
578,6620095,0,KWI5262020,KWI5262020,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
822,3552492,0,LR-FB-CC-177,LR-FB-CC-177,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13258376,977374,3636,GT2500B,GHFJF0322,0.952684,0.047316,3527,0.852285,0.147715,3633,0.436623,0.563377,3533,0.240404,0.759596,1803,0.000617,0.999383,0.371749
13258540,977380,3642,GT2500B,050321-4,0.958198,0.041802,3535,0.855599,0.144401,3642,0.456205,0.543795,3514,0.237223,0.762777,794,0.000232,0.999768,0.368226
13258981,977373,3635,GT2500B,5030220-1,0.952495,0.047505,3586,0.879486,0.120514,3641,0.455532,0.544468,3600,0.260810,0.739190,852,0.000250,0.999750,0.357600
13259201,977378,3640,GT2500B,S102819-1,0.955531,0.044469,3618,0.896879,0.103121,3643,0.457764,0.542236,3631,0.283424,0.716576,1406,0.000452,0.999548,0.347769


In [128]:
%%time
product_similarity = similarity[(similarity['initial_product'].isin(product_list))&(similarity['product_ID'].isin(product_list))]

CPU times: user 730 ms, sys: 10 ms, total: 740 ms
Wall time: 737 ms


In [129]:
product_similarity

,Unnamed: 0,Unnamed: 0_cannabis_perc,initial_product,product_ID,product_cos_distance_cannabis_perc,product_cos_similarity_cannabis_perc,Unnamed: 0_terpenes_perc,product_cos_distance_terpenes_perc,product_cos_similarity_terpenes_perc,Unnamed: 0_cannabis_vol,product_cos_distance_cannabis_vol,product_cos_similarity_cannabis_vol,Unnamed: 0_terpenes_vol,product_cos_distance_terpenes_vol,product_cos_similarity_terpenes_vol,Unnamed: 0_terpenes_cannabis_vol,product_cos_distance_terpenes_cannabis_vol,product_cos_similarity_terpenes_cannabis_vol,weighted_w_tcvol
235,2502732,0,BO.NH,BO.NH,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
289,973738,0,GT2500B,GT2500B,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
514,5355280,0,HS008-HL-042420,HS008-HL-042420,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
578,6620095,0,KWI5262020,KWI5262020,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
822,3552492,0,LR-FB-CC-177,LR-FB-CC-177,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12210647,2374816,3304,LAKKB021420A,KSS040720,0.001122,0.998878,3186,0.632465,0.367535,3213,0.000778,0.999222,3033,0.158379,0.841621,2391,0.000145,0.999855,0.709781
12406439,2372261,749,LAKKB021420A,BO.NH,0.000062,0.999938,3305,0.657722,0.342278,777,0.000053,0.999947,3210,0.170059,0.829941,1572,0.000050,0.999950,0.698204
12406440,2504003,1271,BO.NH,LAKKB021420A,0.000062,0.999938,3487,0.657722,0.342278,1439,0.000053,0.999947,3587,0.170059,0.829941,1484,0.000050,0.999950,0.698204
12790975,2371823,311,LAKKB021420A,GT2500B,0.000019,0.999981,3493,0.721502,0.278498,303,0.000013,0.999987,3604,0.221132,0.778868,3445,0.000766,0.999234,0.666872


In [130]:
product_similarity[product_similarity['initial_product'] == 'HB200422A-SA']

,Unnamed: 0,Unnamed: 0_cannabis_perc,initial_product,product_ID,product_cos_distance_cannabis_perc,product_cos_similarity_cannabis_perc,Unnamed: 0_terpenes_perc,product_cos_distance_terpenes_perc,product_cos_similarity_terpenes_perc,Unnamed: 0_cannabis_vol,product_cos_distance_cannabis_vol,product_cos_similarity_cannabis_vol,Unnamed: 0_terpenes_vol,product_cos_distance_terpenes_vol,product_cos_similarity_terpenes_vol,Unnamed: 0_terpenes_cannabis_vol,product_cos_distance_terpenes_cannabis_vol,product_cos_similarity_terpenes_cannabis_vol,weighted_w_tcvol
2432,4947040,0,HB200422A-SA,HB200422A-SA,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,0,0.000000,1.000000,1.000000
680688,4949789,2749,HB200422A-SA,HS019-HL-063020,0.000367,0.999633,10,0.035414,0.964586,2755,0.000273,0.999727,9,0.006391,0.993609,2976,0.000452,0.999548,0.983708
4187851,4949827,2787,HB200422A-SA,LAKKB021420A,0.000381,0.999619,458,0.161259,0.838741,2920,0.000332,0.999668,1514,0.051947,0.948053,1064,0.000033,0.999967,0.925333
4955774,4948390,1350,HB200422A-SA,LR-FB-CC-177,0.000102,0.999898,894,0.196352,0.803648,1098,0.000056,0.999944,562,0.035125,0.964875,1883,0.000125,0.999875,0.910728
6796538,4947687,647,HB200422A-SA,POG.NH,0.000043,0.999957,1518,0.279431,0.720569,888,0.000043,0.999957,1402,0.049576,0.950424,2535,0.000274,0.999726,0.872982
7204324,4949007,1967,HB200422A-SA,21040521C,0.000180,0.999820,1628,0.299376,0.700624,1726,0.000109,0.999891,1501,0.051672,0.948328,334,0.000003,0.999997,0.864026
7660503,4949721,2681,HB200422A-SA,HS008-HL-042420,0.000345,0.999655,1769,0.321688,0.678312,2835,0.000297,0.999703,1636,0.054850,0.945150,2751,0.000340,0.999660,0.853812
9333149,4948600,1560,HB200422A-SA,KWI5262020,0.000128,0.999872,2316,0.412298,0.587702,1852,0.000124,0.999876,2036,0.065095,0.934905,2890,0.000397,0.999603,0.813016
10760239,4948735,1695,HB200422A-SA,BO.NH,0.000144,0.999856,3008,0.506586,0.493414,1855,0.000124,0.999876,2920,0.089990,0.910010,2089,0.000163,0.999837,0.769776
11384034,4949314,2274,HB200422A-SA,KSS040720,0.000239,0.999761,3301,0.557775,0.442225,1848,0.000123,0.999877,2940,0.091326,0.908674,2677,0.000316,0.999684,0.746896


In [131]:
product_similarity.columns

Index(['Unnamed: 0', 'Unnamed: 0_cannabis_perc', 'initial_product',
       'product_ID', 'product_cos_distance_cannabis_perc',
       'product_cos_similarity_cannabis_perc', 'Unnamed: 0_terpenes_perc',
       'product_cos_distance_terpenes_perc',
       'product_cos_similarity_terpenes_perc', 'Unnamed: 0_cannabis_vol',
       'product_cos_distance_cannabis_vol',
       'product_cos_similarity_cannabis_vol', 'Unnamed: 0_terpenes_vol',
       'product_cos_distance_terpenes_vol',
       'product_cos_similarity_terpenes_vol',
       'Unnamed: 0_terpenes_cannabis_vol',
       'product_cos_distance_terpenes_cannabis_vol',
       'product_cos_similarity_terpenes_cannabis_vol', 'weighted_w_tcvol'],
      dtype='object')

In [132]:
product_similarity = product_similarity[['initial_product','product_ID','weighted_w_tcvol']]


In [133]:
product_similarity_sort = product_similarity.groupby('initial_product')['weighted_w_tcvol'].mean().sort_values(ascending = False)
product_similarity_sort

initial_product
HS008-HL-042420    0.923555
LR-FB-CC-177       0.911770
21040521C          0.910521
HS019-HL-063020    0.903266
KWI5262020         0.901969
POG.NH             0.894334
BO.NH              0.875624
HB200422A-SA       0.860841
KSS040720          0.854492
GT2500B            0.837839
LAKKB021420A       0.833407
Name: weighted_w_tcvol, dtype: float64

In [134]:
product_similarity_sort.head()

initial_product
HS008-HL-042420    0.923555
LR-FB-CC-177       0.911770
21040521C          0.910521
HS019-HL-063020    0.903266
KWI5262020         0.901969
Name: weighted_w_tcvol, dtype: float64

In [135]:
clustroid = product_similarity_sort.head(1).index[0]
clustroid

'HS008-HL-042420'

## Nearest products to the clustroid

In [136]:
%%time
similarity[similarity['initial_product'] == clustroid][['product_ID','weighted_w_tcvol']].head(10)

CPU times: user 409 ms, sys: 5.54 ms, total: 414 ms
Wall time: 411 ms


,product_ID,weighted_w_tcvol
514,HS008-HL-042420,1.000000
22593,LB235,0.997595
26014,GHFSUG0618,0.997394
29597,NB287,0.997208
39692,NB296,0.996737
45395,S120519-1,0.996508
45947,NB183,0.996485
50029,GHFWDC0617,0.996325
58276,GHFWIFI0924SMB,0.996020
59277,NB191,0.995984


In [137]:
clustered.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Strain_Name', 'Batch_ID', 'A_Pinene',
       'Distributor', 'Strain', 'MetricID', 'Type', 'Distributor_Address',
       ...
       'dominant_terpene_encoded', 'second_dominant_terpene_encoded',
       'third_dominant_terpene_encoded', 'dominant_terpene_perc_level_encoded',
       'second_dominant_terpene_encoded_encoded',
       'third_dominant_terpene_encoded_encoded',
       'terpenefirst_multilayer_cluster_full_label',
       'terpenefirst_multilayer_cluster_6digit_label',
       'terpenefirst_cluster_adjust_label_1',
       'terpenefirst_cluster_adjust_label_2'],
      dtype='object', length=105)

In [138]:
cluster_label= clustered[clustered['Batch_ID'] == clustroid][[
       'terpenefirst_cluster_adjust_label_2']].values
cluster_label 


array([['010467167']], dtype=object)

In [139]:
clustered[clustered['terpenefirst_cluster_adjust_label_2'] == '310236236']['Batch_ID']

Series([], Name: Batch_ID, dtype: object)

In [140]:
product_similarity_sort['HS008-HL-042420']


0.9235550133972178

## Factoring ratings

Make the high rate products weight more when looking for the clustroid

In [141]:
new_reviews['rating_norm'].describe()

count    165.000000
mean      -0.002909
std        0.849609
min       -3.160000
25%       -0.160000
50%       -0.160000
75%        0.840000
max        1.500000
Name: rating_norm, dtype: float64

In [142]:
new_reviews[(new_reviews['username'] == 'kasper0017@gmail.com')&(new_reviews['Stimulated'] ==1)]['rating_norm'].sort_values(ascending = False)

126    0.84
53     0.84
114    0.84
62     0.84
57     0.84
61     0.84
124   -0.16
48    -0.16
45    -0.16
41    -0.16
39    -0.16
32    -0.16
113   -0.16
92    -0.16
106   -0.16
99    -0.16
102   -1.16
108   -1.16
13    -1.16
95    -1.16
83    -1.16
63    -1.16
49    -1.16
97    -2.16
66    -2.16
Name: rating_norm, dtype: float64

Option 1: using top percentile

Option 2: using a fixed threshold

In [143]:
product_list = new_reviews[(new_reviews['username'] == 'kasper0017@gmail.com')&(new_reviews['Stimulated'] ==1)]
product_list_high_rating = product_list[product_list['rating_norm'] >=product_list['rating_norm'].quantile(0.6)]

In [144]:
# find the products with top 0.4 percent of ratings
product_list[product_list['rating_norm'] >=product_list['rating_norm'].quantile(0.6)]


,username,Date,Batch ID,Matrix,type,Strain Name,Brand,notes,smokeType,timeOfDay,...,Cottonmouth,Sluggish,Hazy,Annoyed,Aroused,Emotional,Antisocial,rating_mean,rating_norm,review_sentiment
32,kasper0017@gmail.com,2020-12-11 14:24:45,S091620-1,Plant,Flower - Cured,Double Dream,Pure Beauty,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,-0.16,Positive
39,kasper0017@gmail.com,2020-12-11 22:36:22,LCT.NH,Plant,Flower - Cured,Lemon Citrine,The Plug LA,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,-0.16,Positive
41,kasper0017@gmail.com,2020-12-11 22:49:46,LB096,Concentrates & Extracts,Live Resin,Skystomper,Raw Garden,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,-0.16,Positive
45,kasper0017@gmail.com,2020-05-17 22:32:44,RLR050,Concentrates & Extracts,Diamonds,Pink Lemonade,Raw Garden,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,-0.16,Positive
48,kasper0017@gmail.com,2020-05-17 22:35:33,LCT.NH,Plant,Flower - Cured,Lemon Citrine,The Plug LA,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,-0.16,Positive
53,kasper0017@gmail.com,2020-05-17 22:38:39,OZK.NH,Plant,Flower - Cured,OZ Kush,The Plug LA,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,0.84,Positive
57,kasper0017@gmail.com,2020-05-17 22:41:01,RM4H28.OC,Plant,Flower - Cured,Orange Creamsicle,The Plug LA,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,0.84,Positive
61,kasper0017@gmail.com,2020-05-17 22:46:22,RLR050,Concentrates & Extracts,Diamonds,Pink Lemonade,Raw Garden,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,0.84,Positive
62,kasper0017@gmail.com,2020-05-17 22:47:02,RLR049,Concentrates & Extracts,Diamonds,Moonwalker OG,Raw Garden,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,0.84,Positive
92,kasper0017@gmail.com,2020-12-26 17:24:06,9849,Concentrates & Extracts,Live Resin,Arise,Everest,NaN,NaN,NaN,...,0,0,0,0,0,0,0,4.155738,-0.16,Positive


## Combine Scores

In [145]:
new_reviews[new_reviews['Batch ID'].isin(intersect)][effects_experiences]
product_list = intersect

In [146]:
result_df = pd.DataFrame()
for p in tqdm(product_list):
    #product_similarity_sort

    temp = similarity[similarity['initial_product'] == p][['product_ID','weighted_w_tcvol']].head(5)
    if len(temp) >0:
        temp['combined_score'] = temp['weighted_w_tcvol'] * product_similarity_sort[p]
    else:
        continue
    result_df = pd.concat([result_df,temp])   
    
    #print(result_df)

100%|███████████████████████████████████████████| 17/17 [00:07<00:00,  2.33it/s]


In [147]:
final_result = result_df.sort_values('combined_score',ascending=False).drop_duplicates(subset = 'product_ID')

In [148]:
final_result

,product_ID,weighted_w_tcvol,combined_score
514,HS008-HL-042420,1.000000,0.923555
22593,LB235,0.997595,0.921334
26014,GHFSUG0618,0.997394,0.921149
29597,NB287,0.997208,0.920977
39692,NB296,0.996737,0.920541
822,LR-FB-CC-177,1.000000,0.911770
2821,21040521C,1.000000,0.910521
43121,LB353,0.996602,0.908671
70742,210000072,0.995595,0.907754
71633,LB347,0.995565,0.907726


In [149]:
new_reviews.columns

Index(['username', 'Date', 'Batch ID', 'Matrix', 'type', 'Strain Name',
       'Brand', 'notes', 'smokeType', 'timeOfDay', 'rating', 'experiences_',
       'Art / Music', 'Mood Boosting', 'Exercise', 'Focus', 'Gaming',
       'Getting Stoned', 'I didn't like it', 'Creativity', 'Mornings',
       'Pain Relief', 'Relaxing', 'Romancing', 'Sleeping', 'Socializing',
       'effects_', 'Stimulated', 'Calm', 'Creative', 'Energized', 'Euphoric',
       'Focused', 'Relaxed', 'Sleepy', 'Uplifted', 'Sedated', 'othereffects_',
       'Spacey', 'Boring', 'Paranoia', 'Dry eyes', 'Headache', 'Anxious',
       'Dizzy', 'Cottonmouth', 'Sluggish', 'Hazy', 'Annoyed', 'Aroused',
       'Emotional', 'Antisocial', 'rating_mean', 'rating_norm',
       'review_sentiment'],
      dtype='object')

# Function

In [70]:
def most_reviewed_experiences(user):
    #user = 'kasper0017@gmail.com'
    user_mean = new_reviews[new_reviews['username'] == user].groupby('username')[experiences_col].mean().T

    user_mean = user_mean.reset_index().rename(columns = {'index':'experiences',
                                                         user: 'score'})
    user_mean.index = user_mean.index.set_names(['index'])
    user_mean = user_mean.sort_values('score',ascending = False)
    result = user_mean.head(3)['experiences'].values
    return result
def most_reviewed_effects(user):
    #user = 'kasper0017@gmail.com'
    user_mean = new_reviews[new_reviews['username'] == user].groupby('username')[effects_col].mean().T

    user_mean = user_mean.reset_index().rename(columns = {'index':'effects',
                                                         user: 'score'})
    user_mean.index = user_mean.index.set_names(['index'])
    user_mean = user_mean.sort_values('score',ascending = False)
    result = user_mean.head(3)['effects'].values
    return result

**Base products**:

  The list of products that has been reviewed, that matches the search query of users to a certain degree. 
  
  We use base products to find a clustroid (representative) , and use the distance to clustroid to measure the candidates' relevancy (Score1). 
  
  We also go through the products that were clustered to the same cluster as the clustroid, find their weighted cosine similarity to clustroid(Score2)
  
  The final score of the candidates would be score1 times score2

**The rules of selecting base products:**

    1. if user doesn't enter any effects or experiences, return all the products that has been reviewed
    2. if user doesn't enter any effects, but selected experiences, return all the products, that are EXACT match in the experiences 0+all
    3. same for experience as No.2  0+all
    4. if user selected experiences ad effects, return the intersection of  approximate match in experiences and effects. any + any  

In [71]:
data['Distributor'].value_counts()

Central Coast Ag Distribution. LLC               2120
ALKHEMIST DM LLC                                  330
Central Coast Ag Distribution, LLC                300
BSeen LLC                                         192
0                                                 128
CANN Distributors                                 109
Big Oil. Co.                                      100
BSEEN LLC                                          76
420-3. LLC                                         48
SONOMA PACIFIC DISTRIBUTION I                      40
CI Distribution. LLC / DSM Group LLC               29
CA MANUFACTURING SOLUTIONS.                        28
NATURA DISTRIBUTION, LLC                           27
1346 LA, LLC                                       25
PHC Facilities. Inc.                               22
CA MANUFACTURING SOLUTIONS, LLC                    21
The Hive Laboratory                                20
Big Distro                                         12
420-3, LLC                  

In [72]:
def products_rating_filter(intersect):
    temp = new_reviews[new_reviews['Batch ID'].isin(intersect)]
    temp['rating_norm'].sort_values(ascending = False)
    product_list_high_rating = temp[temp['rating_norm'] >=temp['rating_norm'].quantile(0.6)]['Batch ID'].values
    return product_list_high_rating
    
    

## Adding simple retailer filter

There are several options to add the retailer filter in the recommendation function.
1. Before creating similarity matrix (create separate matrix for each of the retailer)
2. Before selecting base products (only the product available at the retailer can be considered as the base products.
3. Before looking up the similarity matrix and finding the similar products of the clustroid. Only the products available at the retailer can be find the similarity matrix. 

The modification below is the third case.

In [73]:

def products_retailer_filter(data,retailer):
    data_retailer_filtered = data[data['Distributor'].isin(retailer)]
    return data_retailer_filtered

In [74]:
def getproducts(user, effects, experiences):
  '''
  'effects' is the user selected effects, list, length can be 0,1,2
  'experiences' is the user selected effects, list, length can be 0,1,2
  '''
  effects_experiences = effects+experiences  
  default_list = set(list(new_reviews['Batch ID'].values))

  ### approximate match (any = 1 in experiences and any = 1 in effects)

  ## consider the case where user doesn't select any

  if len(effects) ==0 and len(experiences) ==0:
    intersect = default_list
  elif len(effects) ==0:
    effect_temp = default_list
    experience_temp = set(list(new_reviews[new_reviews[experiences].isin([1]).all(axis=1)]['Batch ID']))
  elif len(experiences) ==0:
    experience_temp = default_list
    effect_temp = set(list(new_reviews[new_reviews[effects].isin([1]).all(axis=1)]['Batch ID']))
  else:
    experience_temp = set(list(new_reviews[new_reviews[experiences].isin([1]).any(axis=1)]['Batch ID']))
    effect_temp = set(list(new_reviews[new_reviews[effects].isin([1]).any(axis=1)]['Batch ID']))
  

  # the intersection of the two above
  intersect = list(experience_temp.intersection(experience_temp))
  #new_reviews[new_reviews['Batch ID'].isin(intersect)][effects_experiences]
  intersect = products_rating_filter(intersect)
    
  print(f'number of base products is {len(intersect)}')
  return intersect




In [75]:
user = 'kasper0017@gmail.com'
#case 1
# experiences = []
# effects = []

#case2
# effects = ['Euphoric','Focused']
# experiences = ['Sleeping']

#case3
# effects = []
# experiences = ['Sleeping']

#case4
effects = ['Euphoric','Focused']
experiences = []

#case5
# effects = ['Euphoric','Focused']
# experiences = ['Sleeping','Socializing']
effects_experiences = effects+experiences  
effects_experiences 


['Euphoric', 'Focused']

In [76]:
len(getproducts(user, effects, experiences))

number of base products is 70


70

In [77]:
similarity_df = similarity[['initial_product','product_ID','weighted_w_tcvol']]

In [160]:
def user_experience_get_rec(user_input, retailer_input,effects_input, experiences_input):
    """
    This function returns a sorted list of reccmended products based on the desired effects and experiences a user selects
    The rank is based on the final score, which was calculated using the weighted cosine similarity to the clustroid 
    and sum of distance of the clustroid to all the other points. Only products in the same cluster as the clustroid will be considered in the rank
    In this way, the function factors the cosine similarity and the cluster result, balances the absolute distance and some randomness in the unsupervised clustering. 
    """
    
    product_ids = getproducts(user_input, effects_input, experiences_input)
    
    #print(product_ids)
    if len(product_ids) >0:

        product_list =  product_ids
        
        ###****changed
        data_retailer_filtered = products_retailer_filter(data,retailer_input)
        retailer_filtered_ID = list(data_retailer_filtered['Batch_ID'].values)
        ###***changed
        
        product_similarity = similarity_df[(similarity_df['initial_product'].isin(product_list)) &(similarity_df['product_ID'].isin(product_list))]
        product_similarity_sort = product_similarity.groupby('initial_product')['weighted_w_tcvol'].mean().sort_values(ascending = False)
        #print(product_similarity_sort.head())
        clustroid = product_similarity_sort.head(1).index[0]
        print(f'the clustroid: {clustroid}')
        similarity_rec = similarity_df[similarity_df['initial_product'] == clustroid][['product_ID','weighted_w_tcvol']].head(10)['product_ID'].values
        cluster_label= clustered[clustered['Batch_ID'] == clustroid][[
              'terpenefirst_cluster_adjust_label_2']].values[0][0]
#         cluster_label= clustered[clustered['Batch_ID'] == clustroid][[
#               'terpenefirst_cluster_adjust_label_2']].values
        
        print(cluster_label)
        cluster_rec = clustered[clustered['terpenefirst_cluster_adjust_label_2'] == cluster_label]['Batch_ID'].values
    #     print(similarity_rec)
        print(cluster_rec)

        result_df = pd.DataFrame(columns = ['product_ID','weighted_w_tcvol'])
        for p in tqdm(product_list):
            #product_similarity_sort

            temp = similarity_df[(similarity_df['initial_product'] == p)& (similarity_df['product_ID'].isin(retailer_filtered_ID))][['product_ID','weighted_w_tcvol']].head(5)
            if len(temp) >0:
                temp['combined_score'] = temp['weighted_w_tcvol'] * product_similarity_sort[p]
            else:
                continue
            result_df = pd.concat([result_df,temp])
       
        final_result = result_df.sort_values('combined_score',ascending=False).head(100).drop_duplicates(subset = 'product_ID').reset_index()
        final_result = final_result.drop('index', axis = 1)   
   

    else:
        print("Leave a review about a product you used!")
        # direct to baseline recommendation
        # product_list = reviews[reviews['username'] == user].reset_index()
        # product = product_list.loc[0,'Batch ID']

        # similarity_rec =  similarity[similarity['initial_product'] == product][['product_ID','weighted_w_tcvol']].head(10)['product_ID'].values
        # cluster_label= clustered[clustered['Batch_ID'] == product][[
        #   'terpenefirst_cluster_adjust_label_2']].values[0][0]
        # cluster_rec = clustered[clustered['terpenefirst_cluster_adjust_label_2'] == cluster_label]['Batch_ID'].values
        # result_df = pd.DataFrame(columns = ['product_ID','weighted_w_tcvol'])
    
        # temp = similarity[similarity['initial_product'] == product][['product_ID','weighted_w_tcvol']].head(5)
        # temp['combined_score'] = temp['weighted_w_tcvol']
        # result_df = pd.concat([result_df,temp])
    
        # final_result = result_df.sort_values('combined_score',ascending=False).drop_duplicates(subset = 'product_ID').reset_index()
        # final_result = final_result.drop('index', axis = 1)
      
    return final_result

In [161]:
reviews.loc[0,'Batch ID']


'HS019-HL-063020'

In [162]:
%%time
user_experience_get_rec('kasper0017@gmail.com',['Central Coast Ag Distribution. LLC'],['Euphoric','Focused'],['Sleeping'])

number of base products is 17
the clustroid: 21040521C
112
['GHFSSH0923' 'GHFBF0406SMB' 'LDRBF0407PR' '21040521C' 'GHFSSH0514AA'
 'FFMIDBRY0527MP' 'GHFPOG0811SMB' 'GHFRUNTZ0823' 'GHFRUNTZ0902'
 'GHFBCS0922LB' 'GHFCAK0930GSMB']


100%|███████████████████████████████████████████| 17/17 [00:14<00:00,  1.14it/s]

CPU times: user 15.6 s, sys: 107 ms, total: 15.8 s
Wall time: 15.7 s


,product_ID,weighted_w_tcvol,combined_score
0,210000414,0.993407,0.929831
1,RV2000182,0.992442,0.928928
2,210000140,0.992145,0.928649
3,RLR006,0.991317,0.927874
4,210000092,0.990717,0.927313
5,LB235,0.997595,0.924211
6,NB287,0.997208,0.923852
7,NB296,0.996737,0.923415
8,LB220,0.995961,0.922696
9,NB333,0.995600,0.922362


In [163]:
user_experience_get_rec('kasper0017@gmail.com',['Central Coast Ag Distribution. LLC'],['Euphoric','Focused'],[])

number of base products is 70
the clustroid: HS008-HL-042420
010467167
['NB433' 'HS008-HL-042420' '210000299' '210000530']


100%|███████████████████████████████████████████| 70/70 [01:01<00:00,  1.14it/s]


,product_ID,weighted_w_tcvol,combined_score
0,LB235,0.997595,0.879956
1,NB465,1.000000,0.879915
2,NB287,0.997208,0.879614
3,NB296,0.996737,0.879198
4,LB220,0.995961,0.878514
5,NB333,0.995600,0.878196
6,LB471,0.995479,0.875937
7,RV2000118,0.995395,0.875863
8,LB452,0.994631,0.875190
9,LB263,0.994620,0.875181


In [ ]:
#user_experience_get_rec('kasper0017@gmail.com',[],['Focus','Gaming'])

In [ ]:
experiences_col

In [154]:
user_experience_get_rec('kasper0017@gmail.com',['Central Coast Ag Distribution. LLC'],['Aroused',
 'Emotional'],['Mood Boosting'])

number of base products is 27
the clustroid: OZK.NH
320493193
['OZK.NH' 'GHFHF0427' 'GHFFAT0907PR' 'GHFSTAR0921PR']


100%|███████████████████████████████████████████| 27/27 [00:11<00:00,  2.40it/s]


,product_ID,weighted_w_tcvol,combined_score
0,OZK.NH,1.000000,0.944735
1,210000355,0.995112,0.940117
2,NB446,0.994794,0.939817
3,210000109,0.994203,0.939259
4,LB164,0.994162,0.939220
5,NB464,1.000000,0.934634
6,NB423,0.999179,0.933867
7,LB212,0.999127,0.933819
8,LB135,0.998941,0.933644
9,LB150,0.998889,0.933596


In [ ]:
clustered[clustered['Batch_ID'] == 'OZK.NH']

In [155]:
user_experience_get_rec('kasper0017@gmail.com',['Central Coast Ag Distribution. LLC'],['Stimulated',
 'Calm'],[])

number of base products is 70
the clustroid: HS008-HL-042420
010467167
['NB433' 'HS008-HL-042420' '210000299' '210000530']


100%|███████████████████████████████████████████| 70/70 [00:29<00:00,  2.38it/s]


,product_ID,weighted_w_tcvol,combined_score
0,HS008-HL-042420,1.000000,0.882077
1,LB235,0.997595,0.879956
2,NB465,1.000000,0.879915
3,GHFSUG0618,0.997394,0.879779
4,NB287,0.997208,0.879614
5,NB296,0.996737,0.879198
6,NB212,0.997358,0.877591
7,BDRP-3445,1.000000,0.876779
8,LB471,0.995479,0.875937
9,RV2000118,0.995395,0.875863


In [156]:
### testing on new users

user_experience_get_rec('rthakkar@usc.edu',['Central Coast Ag Distribution. LLC'],['Focused'],[])

number of base products is 70
the clustroid: HS008-HL-042420
010467167
['NB433' 'HS008-HL-042420' '210000299' '210000530']


100%|███████████████████████████████████████████| 70/70 [00:29<00:00,  2.38it/s]


,product_ID,weighted_w_tcvol,combined_score
0,HS008-HL-042420,1.000000,0.882077
1,LB235,0.997595,0.879956
2,NB465,1.000000,0.879915
3,GHFSUG0618,0.997394,0.879779
4,NB287,0.997208,0.879614
5,NB296,0.996737,0.879198
6,NB212,0.997358,0.877591
7,BDRP-3445,1.000000,0.876779
8,LB471,0.995479,0.875937
9,RV2000118,0.995395,0.875863


In [ ]:
# Cluster Based

In [ ]:
# def get_recommendation(user, experience):
#     result_dict = {}
#     result_dict_2= {}
#     result_dict_3= {}
#     temp = reviews_cluster[(reviews_cluster['Account'] == user) & (reviews_cluster['Experience'] == experience )][['Batch ID','cluster_terpene_only', 'terpenefirst_sub_cluster',
#    'terpenefirst_sub_cluster2', 'terpenefirst_multilayer_label',
#    'terpenefirst_multilayer_cluster_full_label',
#    'terpenefirst_multilayer_cluster_6digit_label',
#        'terpenefirst_cluster_adjust_label_2']].dropna()
#     #print(temp['Batch ID'])
#     thecluster = temp['terpenefirst_cluster_adjust_label_2'].values
#     #print(thecluster)
#     themode = temp['terpenefirst_cluster_adjust_label_2'].mode().values
#     if len(thecluster) == 1 | len(themode) == 0:
#         result_dict[f'{user}:: {experience}'] = thecluster
#     else:
#         result_dict[f'{user}:: {experience}'] = themode
#    # print(result_dict[f'{user}:: {experience}'])
#     recommended = raw[(raw['terpenefirst_cluster_adjust_label_2'].isin(result_dict[f'{user}:: {experience}']))&
#    (~raw['Batch_ID'].isin(temp['Batch ID'].values))]
#     #print(recommended['Batch_ID'].values)
#     print(f"##Top recommendations found: {len(recommended['Batch_ID'].values)} products")
#     print(list(recommended['Batch_ID'].values))
    
#     if len(recommended['Batch_ID'].values) <=30:
#          ## Second Recommendation   
#         second_labels = temp['terpenefirst_multilayer_cluster_6digit_label'].values
#         second_labels_mode = temp['terpenefirst_multilayer_cluster_6digit_label'].mode().values
#         if len(second_labels) == 1 | len(second_labels_mode) == 0:
#             result_dict_2[f'{user}:: {experience}'] = second_labels
#         else:
#             result_dict_2[f'{user}:: {experience}'] = second_labels_mode
#         #print(result_dict_2[f'{user}:: {experience}'])
#         recommended_2 = raw[(raw['terpenefirst_multilayer_cluster_6digit_label'].isin(result_dict_2[f'{user}:: {experience}']))&
#        (~raw['Batch_ID'].isin(temp['Batch ID'].values))&(~raw['Batch_ID'].isin(recommended['Batch_ID'].values))]
#         print(f"##Second recommendations: { len(recommended_2['Batch_ID'].values)} products" )
#         print(list(recommended_2['Batch_ID'].values))
        
#         #### Third Recommendation
#         if len(recommended['Batch_ID'].values) +len(recommended_2['Batch_ID'].values) <=30:
#             third_labels = temp['terpenefirst_multilayer_label'].values
#             third_labels_mode = temp['terpenefirst_multilayer_label'].mode().values
#             if len(third_labels) == 1 | len(third_labels_mode) == 0:
#                 result_dict_3[f'{user}:: {experience}'] = third_labels
#             else:
#                 result_dict_3[f'{user}:: {experience}'] = third_labels_mode
#            # print(result_dict[f'{user}:: {experience}'])
#             recommended_3 = raw[(raw['terpenefirst_multilayer_label'].isin(result_dict_3[f'{user}:: {experience}']))&(~raw['Batch_ID'].isin(temp['Batch ID'].values))&\
#                                 (~raw['Batch_ID'].isin(recommended['Batch_ID'].values))&\
#                                 (~raw['Batch_ID'].isin(recommended_2['Batch_ID'].values))]
#             #print(recommended_3['Batch_ID'].values)
#             print(f"##Third recommendations: { len(recommended_3['Batch_ID'].values)} products" )
#             print(list(recommended_3['Batch_ID'].values))
#             print(f"Total Recommendations: {len(recommended['Batch_ID'].values) +len(recommended_2['Batch_ID'].values) +len(recommended_3['Batch_ID'].values)} products" )
#         else: recommended_3 = []
#     else:recommended_2 = []
    
    
           
#     return 
# #list(recommended['Batch_ID'].values),list(recommended_2['Batch_ID'].values),list(recommended_3['Batch_ID'].values)